In [1]:
import numpy as np
import pandas as pd
import gurobipy as gp

In [2]:
def create_model(A, sense, b, obj, opt=gp.GRB.MAXIMIZE):
    # creating model
    model = gp.Model()

    # creating variable an setting the constraints
    farm_modx = model.addMVar(A.shape[1])
    farm__mod_con = model.addMConstrs(A, farm_modx, sense, b)

    # setting the objective function
    model.setMObjective(None, obj, 0, sense=opt)

    # restricting gurobi logs
    model.Params.OutputFlag = 0

    # optimizing the function
    model.optimize()
    
    return model

## Q1

In [3]:
# done via graphical method

## Q2

A farmer in Iowa owns 450 acres of land. He is going to plant each acre with wheat or corn. Each acre planted with wheat (corn) yields $2,000 ($3,000) profit, requires three (two) workers, and requires two (four) tons of fertilizer. There are currently 1,000 workers and 1,200 tons of fertilizer available.

 

Formulate and solve this problem using gurobi.

 

Next we want to see What happens to the decision variables and the total profit when the availability of fertilizer varies from 200 tons to 2200 tons in 100-ton increments.

 

At what level of fertilizer does the farmer discontinue producing wheat?  That is, what is the smallest level of available fertilizer that results in no wheat being produced?  Your answer should be one of the 100 ton incremental numbers, like 600 or 1300 or...

In [4]:
# defining constraints
A = np.zeros((3, 2))
A[0, :] = [3, 2]
A[1, :] = [2, 4]
A[2, :] = [1, 1]

b = np.array([1000, 1200, 450])

sense = np.array(['<', '<', '<'])

In [5]:
# defining cost
obj = np.array([2000, 3000])

farm_model = create_model(A, sense, b, obj)

print('Acres of wheat and corn respectively', farm_model.x)
print('Maximized profit', farm_model.objVal)

Academic license - for non-commercial use only - expires 2022-08-21
Using license file C:\gurobi912\gurobi.lic
Acres of wheat and corn respectively [200.0, 200.0]
Maximized profit 1000000.0


<ipython-input-5-0e0f9fa1ef13>:4: DeprecationWarning: Deprecated, use Model.addMConstr() instead
  farm_model = create_model(A, sense, b, obj)


In [6]:
# creating empty data frame to store the data
crop_acres = pd.DataFrame(data=None, columns=['fertilizers', 'wheat', 'corn', 'profit'])

# creating array for fertilizer availability
crop_acres['fertilizers'] = np.arange(200, 2300, 100)

for i in range(len(crop_acres)):
    
    # updating the constrains
    b[1] = crop_acres['fertilizers'].iloc[i]
    print('fertilizer limit - ', b[1])
    
    # creating and running the model
    farm_model = create_model(A, sense, b, obj)
    
    # storing the outcomes
    crop_acres.iloc[i, 1:-1] = farm_model.x
    crop_acres.iloc[i, -1] = farm_model.objVal

fertilizer limit -  200
fertilizer limit -  300
fertilizer limit -  400
fertilizer limit -  500
fertilizer limit -  600
fertilizer limit -  700
fertilizer limit -  800
fertilizer limit -  900
fertilizer limit -  1000
fertilizer limit -  1100
fertilizer limit -  1200
fertilizer limit -  1300
fertilizer limit -  1400
fertilizer limit -  1500
fertilizer limit -  1600
fertilizer limit -  1700
fertilizer limit -  1800
fertilizer limit -  1900
fertilizer limit -  2000
fertilizer limit -  2100
fertilizer limit -  2200


<ipython-input-6-b076c8751b88>:14: DeprecationWarning: Deprecated, use Model.addMConstr() instead
  farm_model = create_model(A, sense, b, obj)


In [7]:
crop_acres

,fertilizers,wheat,corn,profit
0,200,100.0,0.0,200000.0
1,300,150.0,0.0,300000.0
2,400,200.0,0.0,400000.0
3,500,250.0,0.0,500000.0
4,600,300.0,0.0,600000.0
5,700,325.0,12.5,687500.0
6,800,300.0,50.0,750000.0
7,900,275.0,87.5,812500.0
8,1000,250.0,125.0,875000.0
9,1100,225.0,162.5,937500.0


In [8]:
print('Min. fertlizers for no wheat', crop_acres.loc[crop_acres['wheat'] == 0, 'fertilizers'].min())

Min. fertlizers for no wheat 1800


## Q3

Star Oil Company is considering five different investment opportunities. The table below gives the required cash outflows and net present values in millions of dollars.

Star Oil has 40 million available for investment now (time 0); it estimates that one year from now (time 1) 20 million will be available for investment. Star Oil may purchase any fraction of each investment, but no more than 100% of each opportunity. In this case, the cash outflows and NPV are adjusted accordingly.

For example, if Star Oil purchases one-fifth of investment 3, then a cash outflow of 1/5 × 5 = 1 million dollars would be required at time 0, and a cash outflow of 1/5 × 5 = 1 million would be required at time 1. The one-fifth share of investment three would yield an NPV of 1/5 ∗ 16 = 3.2 million dollars. Star Oil wants to maximize the NPV that can be obtained by investing in investments 1-5. Formulate an LP that will help achieve this goal. Assume that any funds leftover at time 0 cannot be used at time 1.

What percentage of opportunity 3 should be Star Oil invest in?  Answer in decimals, so if your answer is 54%, you should input 0.54.  Round 2 to decimal places

In [9]:
A = np.zeros((7, 5))
A[0, :] = [11, 53, 5, 5, 29]
A[1, :] = [3, 6, 5, 1, 34]
A[2, :] = [1, 0, 0, 0, 0]
A[3, :] = [0, 1, 0, 0, 0]
A[4, :] = [0, 0, 1, 0, 0]
A[5, :] = [0, 0, 0, 1, 0]
A[6, :] = [0, 0, 0, 0, 1]

b = np.array([40, 20, 1, 1, 1, 1, 1])

cost = np.array([13, 16, 16, 14, 39])

sense = np.array(['<', '<', '<', '<', '<', '<', '<'])

In [10]:
A, b

(array([[11., 53.,  5.,  5., 29.],
        [ 3.,  6.,  5.,  1., 34.],
        [ 1.,  0.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.,  0.],
        [ 0.,  0.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  0.,  1.]]),
 array([40, 20,  1,  1,  1,  1,  1]))

In [11]:
portfolio = create_model(A, sense, b, cost, opt=gp.GRB.MAXIMIZE)

<ipython-input-11-e072eec6a54f>:1: DeprecationWarning: Deprecated, use Model.addMConstr() instead
  portfolio = create_model(A, sense, b, cost, opt=gp.GRB.MAXIMIZE)


In [12]:
portfolio.x

[1.0, 0.20085995085995084, 1.0, 1.0, 0.2880835380835381]

In [13]:
portfolio.objVal

57.449017199017206

## Q4

In [14]:
A = np.zeros((7, 3))

A[0, :] = [107, 500, 0]
A[1, :] = [105, 500, 0]
A[2, :] = [72, 121, 65]
A[3, :] = [72, 121, 65]
A[4, :] = [1, 0, 0]
A[5, :] = [0, 1, 0]
A[6, :] = [0, 0, 1]

sense = np.array(['<', '>', '<', '>', '<', '<', '<'])

b = np.array([50000, 5000, 2250, 2000, 10, 10, 10])

cost = np.array([0.18, 0.23, 0.05])

diet = create_model(A, sense, b, cost, opt=gp.GRB.MINIMIZE)

<ipython-input-14-7698aa7a1b72>:17: DeprecationWarning: Deprecated, use Model.addMConstr() instead
  diet = create_model(A, sense, b, cost, opt=gp.GRB.MINIMIZE)


In [15]:
diet.x

[1.9444444444444444, 10.0, 10.0]

In [16]:
diet.objVal

3.1500000000000004

In [17]:
print('Vitamin C', A[0, :] @ diet.x)

print('calories', A[2, :] @ diet.x)

Vitamin C 5208.055555555556
calories 2000.0


## Q5

Paper and wood products companies need to define cutting schedules that will maximize the total wood yield of their forests over some planning period. Suppose that a firm with control of 2 forest units wants to identify the best cutting schedule over a planning horizon of 3 years. Forest unit 1 has a total acreage of 2 and unit 2 has a total of 3 acres. The studies that the company has undertaken predict that each acre in unit 1(2) will have 1, 1.3, 1.4 (1, 1.2, 1.6) tons of woods per acre available for harvesting in year 1, 2, 3 respectively. Based on its prediction of economic conditions, the company believes that it should harvest at least 1.2, 1.5, 2 tons of wood in year 1, 2, 3 separately. Due to the availability of equipment and personnel, the company can harvest at most 2, 2, 3 tons of wood in year 1, 2, 3. Find the company’s best cutting strategy that maximizes the total weights of wood. Here discounting of the time value should not be considered.  If some fraction of a forest unit is cut down in year 1, that part of the forest cannot be cut again for the remaining 2 years.  Similarly if some fraction of the forest unit is cut down in year 2 it cannot be cut in year 3.

In year 3, how many acres of forest unit 2 should be cut down?  Round to 2 decimal places.

In [19]:
A = np.zeros((8, 6))

A[0, :] = np.array([2, 0, 0, 3, 0, 0])
A[1, :] = np.array([2, 0, 0, 3, 0, 0])
A[2, :] = np.array([0, 2.6, 0, 0, 3.6, 0])
A[3, :] = np.array([0, 2.6, 0, 0, 3.6, 0])
A[4, :] = np.array([0, 0, 2.8, 0, 0, 4.8])
A[5, :] = np.array([0, 0, 2.8, 0, 0, 4.8])

A[6, :] = np.array([1, 1, 1, 0, 0, 0])
A[7, :] = np.array([0, 0, 0, 1, 1, 1])

sense = np.array(['<', '>', '<', '>', '<', '>', '<', '<'])

b = np.array([2, 1.2, 2, 1.5, 3, 2, 1, 1])

cost = A[0, :] + A[2, :] + A[4, :]

In [20]:
forest_model = create_model(A, sense, b, cost)

<ipython-input-20-06be388921c5>:1: DeprecationWarning: Deprecated, use Model.addMConstr() instead
  forest_model = create_model(A, sense, b, cost)


In [22]:
forest_model.x[5]*3

1.875

In [23]:
forest_model.x

[0.23076923076923084, 0.7692307692307692, 0.0, 0.375, 0.0, 0.625]